# You will have to run batching to be able to run the algorithm attached below
Please bare with my inconsistent coding/naming convention as I am trying to adapt to our batching code space
There are a few unutilized function below and I did recoded a few utility functions that I could have repurposed from the current code base.

## This notebook need to ran from carrot/logistics/batching/logistics/algotest.ipynb

In [1]:
%load_ext memory_profiler
%matplotlib inline
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
import seaborn as sns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from datetime import datetime,timedelta
import imp, os, json, redis, urlparse, requests

import batch_plan
from planning import monster
from services import distance
from planning.common import helpers
from output.status_page import StatusPage
from services.checks import checks
from services import metrics

# Additional dependencies used for rendering status page
import config
from output import common
from planning.common.compute import Compute
import pulp, time
import folium
from bokeh.palettes import Category20, Paired12

# Somemore dependencies
from itertools import permutations, combinations
from planning.v4.delivery_combinations_builder.driving_time_assumptions import DrivingTimeAssumptions as DTA
import pyomo.environ as pe
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

No handlers could be found for logger "rollbar"


In [2]:
%%time
zone_id = 95
# snapshot_at = datetime.utcnow()
snapshot_at = datetime(2018, 8, 20, 14, 0, 0, 0) # 8AM historical to UTC
days = 0
date = snapshot_at + timedelta(days=days) 
plan = batch_plan.BatchPlan(zone_id,date,days,snapshot_at)
plan.include_all_deliveries = True
print snapshot_at
plan.debug=True

2018-08-20 14:00:00
CPU times: user 206 ms, sys: 28.6 ms, total: 235 ms
Wall time: 2.17 s


In [3]:
%%time
plan.load()

INFO:data_fetchers.shifts.shift_availabilities:Getting driver locations for zone 95 and snapshot_at 2018-08-20 14:00:00
INFO:data_fetchers.shifts.driver_locations:Use DB for driver locations - snapshot_at 2018-08-20 14:00:00 for driver_ids [594652, 314664] ...


CPU times: user 5.44 s, sys: 95.3 ms, total: 5.54 s
Wall time: 16.3 s


In [4]:
%%time
monster.run(plan)

deliveries to plan 56 
generating delivery combinations...
88 single combinations
2831 delivery combinations built!
Running ISS Batcher with heuristic: greedy
greedy ISS: 0 handoff combinations
running delivery batcher with algo mip_combination_mip_shift
idle_shopper_ratio 0
346 valid combinations
Running solve_mip_combinations for 328 combinations
Solving MIP with 0.01 optimality gap


/Users/sitewang/development/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
INFO:planning.v4.delivery_batcher.solver:problem status for MIP combinations 1 for zone_id 95
INFO:planning.v4.delivery_batcher.solver:problem status for MIP driver shifts 1 for zone_id 95


Running solve_mip_driver_shifts for 42 batches , objective: total_time
batch_all_combinations? True


services/batch_builder.py:19: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timedelta as pandas.Timedelta
  if isinstance(delta, pd.tslib.Timedelta):
INFO:planning.v4.delivery_batcher.solver:problem status for MIP driver shifts 1 for zone_id 95


Running solve_mip_driver_shifts for 216 batches , objective: total_time
batch_all_combinations? True


...

Running solve_mip_driver_shifts for 494 batches , objective: total_time
batch_all_combinations? True


INFO:planning.v4.delivery_batcher.solver:problem status for MIP driver shifts 1 for zone_id 95


Running solve_mip_driver_shifts for 106 batches , objective: total_time
batch_all_combinations? True


INFO:planning.v4.delivery_batcher.solver:problem status for MIP driver shifts 1 for zone_id 95


Running solve_mip_driver_shifts for 72 batches , objective: total_time
batch_all_combinations? True
CPU times: user 11.7 s, sys: 255 ms, total: 12 s
Wall time: 15 s


In [5]:
metrics.Metrics(plan).metrics
StatusPage(plan).save_to_redis()
checks.Checks(plan).checks

INFO:output.status_page:saving batch plan to batching:monster:95:2018-08-20:plan


{'all': True,
 'bans.store_bans': {'check': True},
 'bans.user_bans': {'check': True},
 'batches.active_wl': {'check': True},
 'batches.alcohol': {'check': True},
 'batches.club_warehouses': {'check': True, 'checks': []},
 'batches.duplicate_deliveries': {'check': True},
 'batches.has_meals_items_in_handoff': {'check': True},
 'batches.overlap_batches': {'check': True},
 'batches.pickup': {'check': True},
 'batches.pickup_only_store': {'check': True},
 'batches.store_opened': {'check': True},
 'batches.unattended': {'check': True},
 'warehouse_assignments.active_wl': {'check': True},
 'warehouse_assignments.store_opened': {'check': True}}

In [6]:
from math import radians, sin, atan2, sqrt, cos
import numpy as np

def e2d(loc1, loc2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = radians(loc1[0]), radians(loc1[1]), radians(loc2[0]), radians(loc2[1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c / 1.6  # To miles
    return distance * 5 # in miles

def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_miles = MILES * c
    return total_miles

def general_haversine(df, start_lat="start_lat", start_lon="start_lon", end_lat="end_lat", end_lon="end_lon"):
    lat_1_rad = np.radians(df[start_lat].astype(float))
    lon_1_rad = np.radians(df[start_lon].astype(float))
    lat_2_rad = np.radians(df[end_lat].astype(float))
    lon_2_rad = np.radians(df[end_lon].astype(float))

    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat/2.0)**2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    haversine_distance = np.clip(3959 * c, -np.inf, np.inf)

    return haversine_distance

# A dummy utility function I wrote on day 2...

In [ ]:
# Utiltiy Functions and I/O management
def write_simple_io(plan):
    # Orders
    input_orders = plan.m_deliveries[['delivery_id',
                                      'window_starts_at',
                                      'window_ends_at',
                                      'warehouse_id',
                                      'order_id',
                                      'end_lat',
                                      'end_lon']]

    # Shoppers
    shoppers_id_snap = pd.Series([i['id'] for i in plan.plan_info['shoppers']])
    input_shoppers_dup = plan.m_driver_shifts[plan.m_driver_shifts['driver_id'].isin(shoppers_id_snap)][['driver_id',
                                                                                                        'home_latitude',
                                                                                                        'home_longitude']]
    # assume missiong lon/la to be zone center
    input_shoppers = input_shoppers_dup.drop_duplicates().dropna()

    # Warehouse
    allowed_warehouses_snap = plan.m_allowable_stores.warehouse_location_id.unique()
    all_warehouses_df = plan.stores_fetcher.warehouse_locations # TODO: combine the next step here?
    allowed_warehouses_df = all_warehouses_df[all_warehouses_df['warehouse_location_id'].isin(allowed_warehouses_snap)]
    input_warehouses = allowed_warehouses_df[['warehouse_location_id',    # unique record
                                              'warehouse_id',             # for retailers
                                              'latitude',
                                              'longitude']]

    # Output .csv files that contains all necessary data for external solvers
    file_id = "zone{0}_{1}".format(plan.zone_id, str(plan.snapshot_at).replace(' ', '_'))
    input_orders.to_csv("{0}/localdata/batchplan_algo/orders_{1}.csv".format(os.path.expanduser("~"), file_id))
    input_shoppers.to_csv("{0}/localdata/batchplan_algo/shoppers_{1}.csv".format(os.path.expanduser("~"), file_id))
    input_warehouses.to_csv("{0}/localdata/batchplan_algo/warehouses_{1}.csv".format(os.path.expanduser("~"), file_id))

    print "[REPORT] {0} ORDERS || {1} SHOPPERS || {2} WAREHOURSES".format(len(input_orders), len(input_shoppers), len(input_warehouses))
    pass

# Local Algorithm Module Development

Use the traditional development structure: Input -- Problem -- Algo -- Solver -- Output

## Algorithm Development Plan: Exact algorithms & Heuristics
* SPP + Assignment 

Ideally, `problem()` defines a general problem that need to be solved or analyzed

In [10]:
class problem(object):
    def __init__(self, plan):
        self.load_data(plan)
        pass
    
    def load_data(self, plan):
        """ Extract only the needed information from the existing data structure """
        
        # Extract orders information
        order_cols = ['delivery_id','window_starts_at','window_ends_at','warehouse_id','order_id','end_lat','end_lon']
        self.order = plan.m_deliveries[order_cols]
        
        # Extract shopper information
        shopper_cols = ['driver_id','home_latitude','home_longitude', 'available_latitude', 'available_longitude']
        shopper_ids = pd.Series([i['id'] for i in plan.plan_info['shoppers']])
        self.shopper = plan.m_driver_shifts[plan.m_driver_shifts['driver_id'].isin(shopper_ids)][shopper_cols]
        self.shopper['home_latitude'] = self.shopper.home_latitude.fillna(plan.zone.latitude)
        self.shopper['home_longitude'] = self.shopper.home_longitude.fillna(plan.zone.longitude)
        self.shopper['driver_shift_id'] = self.shopper.index
        self.shopper = self.shopper.drop_duplicates(subset=shopper_cols)
        self.shopper = self.shopper.set_index('driver_id', verify_integrity=True)
        self.shopper['driver_id'] = self.shopper.index
        
        # Extract shifts information
        shifts_cols = ['driver_shift_id', 'shift_type', 'driver_id','starts_at','ends_at', 'available_latitude', 'available_longtiude']
        self.shifts = plan.m_driver_shifts[plan.m_driver_shifts['driver_id'].isin(shopper_ids)][shopper_cols]        
        
        # Extract shopper information
        warehouse_snap = plan.m_allowable_stores.warehouse_location_id.unique()
        warehouse_df = plan.stores_fetcher.warehouse_locations
        allowed_warehouse_df = warehouse_df[warehouse_df['warehouse_location_id'].isin(warehouse_snap)]
        warehouse_cols = ['warehouse_location_id','warehouse_id','latitude','longitude','zip_code']
        self.warehouse = allowed_warehouse_df [warehouse_cols]
        
        print "%s orders | %s shoppers | %s warehouse" % (len(self.order), len(self.shopper), len(self.warehouse))
        
        # Attach the plan object
        self.plan = plan
        pass
    
class configuration(object):
    def __init__(self, pairs):
        for key, value in pairs.iteritems():
            setattr(self, key, value)

Ideally, `algorithm` defines an algorithm cell that can be used to solves a `problem`. Note the "Ideally". The following code doesn't obey this idea... ^-^

In [11]:
class LocalAlgorithm(object):
    def __init__(self, prob, user_config={"MAX_BATCH_SIZE": 2, "PACKAGE":"pyomo"}):
        self.prob = prob
        self.columns = None
        self.compute = Compute(prob.plan)
        self.config = configuration(user_config)
        self._set = {} 
        self._var = {} 
        self._param = {}
        self._m = None
        pass
    
    def preprocess(self):
        """ Populate some predetermined data matrix for efficient usage """
        # construct dictionary for fetching nearest warehouse to any delivery
        self.nearest_warehouse = {}
        for i in self.prob.order.itertuples():
            fea_warehouse = self.prob.warehouse[self.prob.warehouse.warehouse_id == i.warehouse_id]
            dist = haversine(i.end_lat, i.end_lon, fea_warehouse['latitude'], fea_warehouse['longitude'])
            self.nearest_warehouse[i.delivery_id] = dist.idxmin()
        pass
    
    def _find_nearest_shopper(self, w_lat, w_lon, evaluator=haversine):
        shopper = self.prob.shopper
        dist = evaluator(float(w_lat), float(w_lon), shopper.home_latitude, shopper.home_longitude)
        return dist.idxmin()
    
    def solve(self):
        """ Top-level Algorithm Wrapper """
        self.preprocess()
        self.enumerate_columns()
        self.optimize_mip()
        self.get_model_output()
        pass
    
    def optimize_mip(self):
        # Construct model using PuLP/Pyomo
        build_st = time.time()
        self._m = self._build_model()
        build_time = time.time() - build_st
        print "took %s seconds to construct mip model" % build_time
        solve_st = time.time()
        self._solve_model()
        solve_time = time.time() - solve_st
        print "took %s seconds to solve mip model" % solve_time
        pass
    
    def enumerate_columns(self):
        # construct new combinations
        st = time.time()
        df = self._populate_routes()
        duration = time.time() - st
        print "[%s] initialize routes" % round(duration, 2)
        
        st = time.time()
        df = self._populate_distance_columns(df)
        duration = time.time() - st
        print "[%s] routes distances" % round(duration, 2)
        
        st = time.time()
        df = self._populate_misc_columns(df)
        duration = time.time() - st
        print "[%s] routes misc " % round(duration, 2)
        
        st = time.time()
        df = self._populate_delivery_time_columns(df)
        duration = time.time() - st
        print "[%s] route delivery time" % round(duration, 2)
        
        st = time.time()
        df = self._populate_presumed_driving_time_columns(df)
        duration = time.time() - st
        print "[%s] route presumed driving time" % round(duration, 2)
                
        st = time.time()
        self.columns = self._filter_delivery_window_revert_disjoint(df)
        duration = time.time() - st
        print "[%s] routes filter R1" % round(duration, 2)
        
        # No more df beyond this line ...........
        
        # Adding Shopper as a Second Dimention (Legacy parameter matrix)
        st = time.time()
        self._d = self._populate_d_matrix(df).values
        duration = time.time() - st
        print "[%s] d matrix df" % round(duration, 2)
        
        st = time.time()
        self.DPT = self._populate_picking_time_matrix(self.columns)
        duration = time.time() - st
        print "[%s] DPT matrix" % round(duration, 2)
        
        st = time.time()
        self.DDT = self._populate_driving_time_matrix(self.columns)
        duration = time.time() - st
        print "[%s] DDT matrix" % round(duration, 2)
        
        st = time.time()
        self.AssignableShopperMatrix = self._filter_route_time_window()
        duration = time.time() - st
        print "[%s] time window filter" % round(duration, 2)
        
        st = time.time()
        self._filter_inference_cleaning()
        duration = time.time() - st
        print "[%s] inference filter" % round(duration, 2)
        
        st = time.time()
        self._other_ugly_misc_steps()
        duration = time.time() - st
        print "[%s] ugly steps..." % round(duration, 2)
        
        print "sending %s routes to solver" % len(self.columns)
        return
    
    def _populate_nearest_shopper_columns(self, df):
        df['loc_0_to_drop'] = list(zip(df.lat_0, df.lon_0))
        df['driver_id'] = df['loc_0_to_drop'].map(lambda x: self._find_nearest_shopper(x[0], x[1]))
        df = df.drop(columns=['loc_0_to_drop'])
        driver_cols = ['driver_id', 'home_latitude', 'home_longitude']
        df = df.merge(self.prob.shopper[driver_cols], on='driver_id', how='left')
        df = df.rename(columns={'driver_id':'nearest_driver', 
                                'home_latitude':'nearest_driver_lat',
                                'home_longitude':'nearest_driver_lon'})
        return df
    
    def _populate_routes(self):
        """populate the deliveries assigned, coroesspoding locations, and sequential distances"""
        cols_dict = {}
        size_dict = {}
        unique_warehouse_ids = self.prob.order.warehouse_id.unique()
        for w in unique_warehouse_ids: # walk through each warehouse_id
            order_group = self.prob.order[self.prob.order.warehouse_id == w]['delivery_id']
            for i in range(1, self.config.MAX_BATCH_SIZE+1):
                for j in self.config.ENUMERATOR(order_group, i):
                    cols_dict[len(cols_dict)] = list(j) + [np.nan]*(self.config.MAX_BATCH_SIZE - len(j))
                    size_dict[len(size_dict)] = len(j)
        cols_df = pd.DataFrame.from_dict(cols_dict, orient='index')
        size_df = pd.DataFrame.from_dict(size_dict, orient='index')
        cols_df['batch_size'] = size_df[0]
        cols_df['column_id'] = cols_df.index
        print "generation before processing %s" % len(cols_df)
        return cols_df
    
    def _populate_distance_columns(self, cols_df):
        for i in range(1, self.config.MAX_BATCH_SIZE+1):
            cols_df = cols_df.rename(columns={i-1: 'delivery_id'}) # mis-match in index, reserve 0 for warehouse assignment
            cols_df = cols_df.merge(self.prob.order[['delivery_id', 'window_starts_at', 'window_ends_at', 'end_lat', 'end_lon']], on="delivery_id", how="left")
            cols_df = cols_df.rename(columns={'delivery_id':'delivery_id_%s' % i, 
                                              'end_lat':'lat_%s' % i, 
                                              'end_lon':'lon_%s' % i,
                                              'window_starts_at': 'open_%s' % i,
                                              'window_ends_at':  'due_%s' % i,
                                              'order_id':'order_id_%s' % i})
            if i > 1: # after the first stop
                cols_df['distance_%s_to_%s' % (i-1, i)] = general_haversine(cols_df, 
                                                                               start_lat='lat_%s' % (i-1), 
                                                                               start_lon='lon_%s' % (i-1), 
                                                                               end_lat='lat_%s' % i, 
                                                                               end_lon='lon_%s' % i)
        # populate the warehouse_id and pick a starting warehouse
        cols_df = cols_df.rename(index=str, columns={'delivery_id_1':'delivery_id'}) # delivery_0 is the first one
        cols_df = cols_df.merge(self.prob.order[['delivery_id', 'warehouse_id']], on='delivery_id', how='left')
        cols_df = cols_df.rename(index=str, columns={'delivery_id':'delivery_id_1'}) # switch back
        cols_df['warehouse_location_id'] = cols_df["delivery_id_1"].map(lambda x:self.nearest_warehouse[x])
        cols_df['warehouse_location_id'] = cols_df.warehouse_location_id.astype('int64') # regulate data type for next merge
        wh_subset_cols = ['warehouse_location_id','latitude','longitude','zip_code']
        cols_df = cols_df.merge(self.prob.warehouse[wh_subset_cols], on="warehouse_location_id", how="left")
        cols_df = cols_df.rename(index=str, columns={'latitude':'lat_0', 'longitude':'lon_0', 'zip_code':'end_zip_code_0'})
        cols_df['distance_0_to_1'] = general_haversine(cols_df, 
                                                           start_lat='lat_0', 
                                                           start_lon='lon_0',
                                                           end_lat='lat_1', 
                                                           end_lon='lon_1')
        cols_df['total_distance'] = cols_df[['distance_%s_to_%s' % (i, i+1) for i in range(self.config.MAX_BATCH_SIZE)]].sum(axis=1)
        return cols_df
    
    def _build_model(self):
        """ 
            Construct a MIP model as following:
            min  cx + dy
            s.t. Ax = 1
                 By >= x for all 
            where,  x_i determines the optimal route options, 
                    y_ij determine the assignment options for optimal route,
                    c_i is the cost vector for routes
                    d_ij is the cost matrix for assignments
                    A_ki is the matrix for SPP problem associated with routes
                    B_ij is the matrix for assignment problem assocaited
            ---------------------------------------------------------------------
            i is used for indexing route columns
            j is used for indexing available shoppers
            k is used for indexing orders
        """
        if self.config.PACKAGE == "pulp":
            m = self._pulp_init_model()
            self._pulp_post_parameters(m)                      # SETS and PARAMETERS
            self._pulp_post_variables(m)                       # VARIABLES
            self._pulp_post_objective(m)                       # OBJECTIVE
            self._pulp_post_delivery_once_constraints(m)       # CONSTRAINTS : require each package to be delivered once
            self._pulp_post_assign_to_shopper_constraints(m)   # CONSTRAINTS : assign each route to a shopper
            self._pulp_post_assign_once_constraints(m)         # CONSTRAINTS : each shopper can only be assigned with one route
        elif self.config.PACKAGE == "pyomo":
            m = self._pyomo_init_model()
            self._pyomo_post_parameters(m)
            self._pyomo_post_model(m)
        return m
    
    def _pulp_init_model(self):
        m = pulp.LpProblem("spp_assignment", pulp.LpMinimize)
        return m
    
    def _pyomo_init_model(self):
        m = pe.ConcreteModel()
        return m
            
    def get_model_output(self):
        """ parse solution output for checking and next steps """
        
        if self.config.PACKAGE == "pulp":
            y_val = cell._var['y']
            assignments = [i for i in y_val if y_val[i].value() == 1.0]
        elif self.config.PACKAGE == "pyomo":
            y_val = self._m.y
            assignments = [i for i in y_val if y_val[i].value == 1.0]
        
        self._output = pd.DataFrame({'driver_id':[i[1] for i in assignments],
                                     'column_id':[i[0] for i in assignments]})
        self.columns["column_id"] = self.columns.index
        column_collection = ["delivery_id_%s" % i for i in range(1,self.config.MAX_BATCH_SIZE+1)] 
        column_collection += ["lat_%s" % i for i in range(self.config.MAX_BATCH_SIZE+1)]
        column_collection += ["lon_%s" % i for i in range(self.config.MAX_BATCH_SIZE+1)]
        column_subset = self.columns[column_collection]
        column_subset['warehouse_id'] = self.columns.warehouse_id.astype('int64')
        column_subset['column_id'] = self.columns.index.astype('int64')
        self._output = self._output.merge(column_subset, on="column_id", how="left")
        self._output = self._output.merge(self.prob.shopper[["driver_id", "available_latitude", "available_longitude"]], on="driver_id", how="left")
        pass
        
    def _populate_misc_columns(self, df):
        df["vehicle_type"] = "car"
        cols = ["delivery_id",
                "geohash", 
                "address_type",
                "window_ends_at", 
                "window_starts_at",
                "inventory_warehouse_location_id",
                "items_count",
                "items_qty",
                "normal_items_count",
                "normal_items_qty",
                "looseweight_items_count",
                "looseweight_items_qty",
                "variable_items_count",
                "variable_items_qty",
                'departments_count',
                'special_request_count', 
                'ordered_weight',
                "end_zip_code"]
        sub = self.prob.plan.m_deliveries[cols]
        for i in range(1, self.config.MAX_BATCH_SIZE+1):
            df = df.rename(index=str, columns={"delivery_id_%s" % i:"delivery_id"})
            df = df.merge(sub, on="delivery_id", how="left")
            df = df.rename(columns={"delivery_id": "delivery_id_%s" % i,
                                    "geohash": "geohash_%s" % i,
                                    "address_type": "address_type_%s" % i,
                                    "window_starts_at": "window_starts_at_%s" % i,
                                    "window_ends_at": "window_ends_at_%s" % i,
                                    "end_zip_code": "end_zip_code_%s" % i,
                                    "inventory_warehouse_location_id": "inventory_warehouse_location_id_%s" % i,
                                    "items_count": "items_count_%s" % i,
                                    "items_qty": "items_qty_%s" % i,
                                    "normal_items_count": "normal_items_count_%s" % i,
                                    "normal_items_qty": "normal_items_qty_%s" % i,
                                    "looseweight_items_count": "looseweight_items_count_%s" % i,
                                    "looseweight_items_qty": "looseweight_items_qty_%s" % i,
                                    "variable_items_count": "variable_items_count_%s" % i,
                                    "variable_items_qty": "variable_items_qty_%s" % i,
                                    "departments_count": "departments_count_%s" % i,
                                    "special_request_count": "special_request_count_%s" % i,
                                    "ordered_weight": "ordered_weight_%s" % i})
            df["dow_%s" % i] = self.prob.plan.dow
            df["hour_%s" % i] = pd.to_datetime(df["window_starts_at_%s" % i].values).tz_localize('UTC').tz_convert(self.prob.plan.local_time_zone).hour
            if i == 1:
                df["store_to_delivery_%s_to_%s" % (i-1, i)] = True
            else:
                df["store_to_delivery_%s_to_%s" % (i-1, i)] = False

        df['items_count'] = df[helpers.columns("items_count", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['items_qty'] = df[helpers.columns("items_qty", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['normal_items_count'] = df[helpers.columns("normal_items_count", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['normal_items_qty'] = df[helpers.columns("normal_items_qty", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['looseweight_items_qty'] = df[helpers.columns("looseweight_items_qty", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['looseweight_items_count'] = df[helpers.columns("looseweight_items_count", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['variable_items_count'] = df[helpers.columns("variable_items_count", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['variable_items_qty'] = df[helpers.columns("variable_items_qty", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['departments_count'] = df[helpers.columns("departments_count", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['special_request_count'] = df[helpers.columns("special_request_count", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        df['ordered_weight'] = df[helpers.columns("ordered_weight", self.config.MAX_BATCH_SIZE)].sum(axis=1)
        return df
    
    def _populate_presumed_driving_time_columns(self, df):
        return DTA(plan=self.prob.plan).compute_driving_time(df)
    
    def _populate_d_matrix(self, routes):
        df = routes[['warehouse_location_id', 'lat_0', 'lon_0']].copy()
        for s in self.prob.shopper.itertuples():
            df['shopper_lat'] = s.home_latitude
            df['shopper_lon'] = s.home_longitude
            df['shopper_%s_init_distance' % s.Index] = general_haversine(df,start_lat='lat_0',
                                                                               start_lon='lon_0', 
                                                                               end_lat='shopper_lat',
                                                                               end_lon='shopper_lon')
        return df.drop(columns=['warehouse_location_id', 'lat_0', 'lon_0', 'shopper_lat', 'shopper_lon'])
    
    def _populate_summary_time_columns(self, df):
        """ Total time comsumption that is independent from the shopper """
        time_cols = ["total_delivery_time", "total_driving_time"]
        df["total_time"] =  df[time_cols].sum(axis=1)
        return df
    
    def _filter_delivery_window_revert_disjoint(self, routes, open_col_name="open", due_col_name="due"):
        """ R1
            Rule out a type of definite infeasible subroutes that has delivery i -> j 
            where start_i > due_j
        """
        if self.config.RELAX_TIME_WINDOW:
            return routes  # omit time window constraints
        to_drop = pd.Series(False, index=routes.index)  # construct filter columns
        for i in range(1, self.config.MAX_BATCH_SIZE):
            to_drop = (routes["%s_%s" % (open_col_name, i)] > routes['%s_%s' % (due_col_name, i+1)]) | to_drop 
        to_drop_cnt, to_drop_perc = to_drop.sum(), round(float(to_drop.sum())/len(routes) * 100)
        print "%s (%s %%) routes dropped [obvious time window infeasible]" % (to_drop_cnt, to_drop_perc)
        return routes[~to_drop]
    
    def _filter_quicksilver_shopper_window(self, routes, open_col_name="open", due_col_name="due", start_time=None):
        """ R2
            Rule out time infeasible subroutes even when the shopper is Quicksilver (60 miles/hour = 1 mile/minute, coefficient 1.0)
            where examination are performed at each delivery
            shopper_loc -> (lat_0, lon_0) -> (lat_0, lon_0)
        """
        if self.config.RELAX_TIME_WINDOW:
            return routes  # omit time window constraints
        to_drop = pd.Series(False, index=routes.index)
        if not start_time: 
            start_time = self.prob.plan.snapshot_at
        routes["quicksilver_arrive_at_0"] = start_time + pd.to_timedelta(routes['driving_time'], unit='m')
        for i in range(1, self.config.MAX_BATCH_SIZE):
            routes["quicksilver_arrive_at_%s" % i] = routes["quicksilver_arrive_at_%s" % (i-1)] + pd.to_timedelta(routes['delivery_time_%s' % i], unit='m')
            to_drop = (routes["quicksilver_arrive_at_%s" % i] > routes['%s_%s' % (due_col_name, i)]) | to_drop
        to_drop_cnt, to_drop_perc = to_drop.sum(), round(float(to_drop.sum())/len(routes) * 100)

        print "%s (%s %%) routes dropped [quicksilver_shopper]" % (to_drop_cnt, to_drop_perc)
        routes = routes.drop(columns=[i for i in routes.columns if 'quicksilver' in i])
        return routes[~to_drop]
    
    def _solve_model(self):
        """
            Template solving process
            should be able to take in any solvers, solver options, and output checkers
        """
        if self.config.PACKAGE == "pulp":
            if self.config.MIP_SOLVER == "cplex":
                self._m.solve(pulp.solvers.CPLEX_CMD(msg=1))
            elif self.config.MIP_SOLVER == "cbc":
                self._m.solve(pulp.solvers.PULP_CBC_CMD(msg=1))
            print '-------------'
            print 'mip status %s' % pulp.LpStatus[self._m.status]
            print "objective = %s" % self._m.objective.value()
        elif self.config.PACKAGE == "pyomo":
            if self.config.MIP_SOLVER == "cplex":
                opt = SolverFactory('cplex')
                opt.options['preprocessing presolve'] = 'y'
            elif self.config.MIP_SOLVER == "cbc":
                opt = SolverFactory('cbc')
            res = opt.solve(self._m, logfile='localAlgo.log')
            print '-------------'
            print 'mip status %s' % res.solver.status
            print "objective = %s" % pe.value(self._m.objective)
            
        pass
    
    def _populate_picking_time_matrix(self, df):
        """ Populate picking time columns """
        
        route = df.copy() # Make a copy of the column sheet
        original_indexer = route.index
        
        shift_cols = ["driver_id", "new_driver", "shopping_mpi", "total_mpi"]
        shifts_df = self.prob.plan.m_driver_shifts[shift_cols].drop_duplicates()
        route = self.compute.get_picking_time_features(route) # once
        
        route["dow"] = self.prob.plan.dow
        route["hour"] = self.prob.plan.snapshot_hour_local
        route["batch_type"] = "delivery"
        
        keep_cols = []
        for i in self.prob.shopper.itertuples():
            route['driver_id'] = i.Index
            route = route.merge(shifts_df, on='driver_id', how="left")
            route = route.merge(self.prob.plan.shifts_fetcher.warehouse_location_mpi,
                                on=["driver_id", "warehouse_location_id"], 
                                how="left", 
                                suffixes=("","_wl"))
            route['total_mpi_wl'] = route.total_mpi_wl.fillna(route.total_mpi) # assumption here
            route["shopping_mpi_wl"] = route.shopping_mpi_wl.fillna(route.shopping_mpi)
            route = route.drop(["total_mpi", "shopping_mpi"], axis=1)
            route = route.rename(columns = {"total_mpi_wl": "total_mpi", 
                                            "shopping_mpi_wl": "shopping_mpi"})
            
            picking_time, _, _ = self.prob.plan.picking_model.predict_df(route, include_cbt_time=True)
            route["picking_time_%s" % i.Index] = picking_time
            keep_cols.append("picking_time_%s" % i.Index)
            route = route.drop(columns=["total_mpi", "shopping_mpi", "driver_id", "new_driver"])
        
        route.index = original_indexer
        route = route.drop(columns=[i for i in route.columns if i not in keep_cols])
        return route
    
    def _populate_delivery_time_columns(self, df):
        """ docstring """
        for i in range(1,self.config.MAX_BATCH_SIZE+1):
            cols_mapping = {"start_lat": "lat_%s" % (i-1), 
                            "start_lon": "lon_%s" % (i-1), 
                            "end_lat": "lat_%s" % i, 
                            "end_lon": "lon_%s" % i,
                            "normal_items_count": "normal_items_count_%s" % i, 
                            "distance": "distance_%s_to_%s" % (i-1, i),
                            "store_to_delivery": "store_to_delivery_%s_to_%s" % (i-1,i),
                            "delivery_started_at": "window_starts_at_%s" % max(1, i),
                            "address_type": "address_type_%s" % max(1, i),
                            "end_zip_code": "end_zip_code_%s" % max(1, i),
                            "dow": "dow_%s" % max(1, i),
                            "hour": "hour_%s" % max(1, i)}
            sub_df = df[df['delivery_id_%s' % i].notnull()]
            sub_idx = sub_df.column_id
            (p, lbuffer, ubuffer, multi) = self.prob.plan.delivery_model.predict_df(sub_df, cols_mapping=cols_mapping)
            delivery_predict = pd.DataFrame({"column_id": sub_idx,
                                             "delivery_time_%s" % i: p,
                                             "delivery_multiple_%s" % i: multi})
            df = df.merge(delivery_predict, on="column_id", how="left")
        delivery_time_cols = helpers.columns("delivery_time", self.config.MAX_BATCH_SIZE)
        df['total_delivery_time'] = df[delivery_time_cols].sum(axis=1)
        return df
    
    def _populate_driving_time_matrix(self, df):
        """ populate columns of driving time for each potential shopper on each route """
        route = df.copy()
        route["dow"] = self.prob.plan.dow
        route["hour"] = self.prob.plan.snapshot_hour_local
        route["batch_type"] = "delivery"
        keep_cols = []
        for i in self.prob.shopper.itertuples():
            route["lat_d"], route["lon_d"] = i.available_latitude, i.available_longitude
            route["distance_d_to_0"] = general_haversine(route, "lat_0", "lon_0", "lat_d", "lon_d")
            pt, _, _ = self.prob.plan.driving_model.predict_df(route, {"distance": "distance_d_to_0"})
            route["driving_time_%s" % i.Index] = pt
            keep_cols.append("driving_time_%s" % i.Index)
            route = route.drop(columns=["distance_d_to_0", "lat_d", "lat_d"])
            
        route = route.drop(columns=[i for i in route.columns if i not in keep_cols])

        return route
    
    def _pulp_post_parameters(self, m):        
        """ Constructing sets and parameters for optimization model """
        
        self._set['column_range'] = self.columns.index
        self._set['shopper_range'] = self.prob.shopper.index
        
        # populate the a matrix for spp problem
        route_assignment_matrix = {}
        delivery_mapping = {i:[] for i in self.prob.order['delivery_id'].tolist()}
        for si, i in enumerate(self.columns.itertuples()):
            for j in range(1, self.config.MAX_BATCH_SIZE+1):
                delivery_id = getattr(i, 'delivery_id_%s' % j)
                if not np.isnan(delivery_id):
                    delivery_mapping[delivery_id].append(int(i.Index))
            for sj, j in enumerate(self.prob.shopper.itertuples()):
                if self.config.RELAX_TIME_WINDOW or self.AssignableShopperMatrix[si, sj]:
                    route_assignment_matrix[int(i.Index), int(j.Index)]= 1.0
        
        self._set['delivery_mapping'] = delivery_mapping
        self._set['route_assignment_matrix'] = route_assignment_matrix
        
        # Costs vectors/matrices
        if self.config.OBJECTIVE == "distance":
            self._param['c'] = self.columns.total_distance
            self._param['d'] = self._d
        elif self.config.OBJECTIVE == "time":
            self._param['c'] = self.columns.total_delivery_time
            self._param['d'] = self.DDT.drop(columns=["column_id"]).values + self.DPT.drop(columns=["column_id"]).values
            
        pass
    
    def _pulp_post_variables(self, m):
        """ template variables """
        cr = self._set['column_range']
        ad = self._set['route_assignment_matrix']
        self._var['x'] = pulp.LpVariable.dicts('x', cr , 0, 1, cat=pulp.LpBinary)
        self._var['y'] = pulp.LpVariable.dicts('y', [(i, j) for (i, j) in ad.keys()], 0, 1, cat=pulp.LpBinary)
        
        pass
    
    def _pulp_post_objective(self, m):
        """ objective function: total costs = route costs + assignment-dependnet costs """
        sr = self._set['shopper_range']
        cr = self._set['column_range']
        ad = self._set['route_assignment_matrix']
        c = self._param['c']
        d = self._param['d']
        x = self._var['x']
        y = self._var['y']
        
        # Need to be careful with the indexer
        m += pulp.lpSum(c[i] * x[i] for i in cr) + pulp.lpSum(d[k,l] * y[(i,j)] for (k,i) in enumerate(cr) for (l,j) in enumerate(sr) if (i,j) in ad.keys())
        pass
    
    
    def _pulp_post_delivery_once_constraints(self, m):
        """ template constraints """
        x = self._var['x']
        delivery_mapping = self._set['delivery_mapping']
        
        for i in self.prob.order.itertuples():
            m += pulp.lpSum(x[j] for j in delivery_mapping[i.delivery_id]) == 1
        pass
    
    def _pulp_post_assign_to_shopper_constraints(self, m):
        """ template CONSTRAINTS : assign each route to a shopper """
        cr = self._set['column_range']
        sr = self._set['shopper_range']
        ad = self._set['route_assignment_matrix']
        
        x = self._var['x']
        y = self._var['y']
        
        for i in cr:
            m += pulp.lpSum(y[i,j] for j in sr if (i,j) in ad.keys()) >= x[i]
        
        pass
    
    def _pulp_post_assign_once_constraints(self, m):
        """ template constraints """
        cr = self._set['column_range']
        sr = self._set['shopper_range']
        ad = self._set['route_assignment_matrix']
        
        y = self._var['y']
        
        for j in sr:
            m += pulp.lpSum(y[i,j] for i in cr if (i,j) in ad.keys()) <= 1
        
        pass
    
    def _pyomo_post_parameters(self, m):
        
        m.ColumnRange = pe.Set(initialize=self.columns.index)
        m.ShopperRange = pe.Set(initialize=self.prob.shopper.index)
        m.OrderRange = pe.Set(initialize=self.prob.order.index)
        
        driving_time = self.DDT.values
        picking_time = self.DPT.values
        
        # populate the a matrix for spp problem
        routeorder_matrix = {}
        routeshopper_cost_matrix = {}
        cost_matrix = {}
        routeassignment_matrix = {}
        routeassignment_list = {i:[] for i in self.columns['column_id'].tolist()}
        routeorder_mapping = {i:[] for i in self.prob.order['delivery_id'].tolist()}
        
        for si, i in enumerate(self.columns.itertuples()):
            for sj, j in enumerate(range(1, self.config.MAX_BATCH_SIZE+1)):
                delivery_id = getattr(i, 'delivery_id_%s' % j)
                if not np.isnan(delivery_id):
                    routeorder_matrix[int(i.Index), delivery_id]=1.0
                    routeorder_mapping[delivery_id].append(int(i.Index))
                    
            for sj, j in enumerate(self.prob.shopper.itertuples()):
                if self.config.RELAX_TIME_WINDOW or self.AssignableShopperMatrix[si, sj]:
                    routeassignment_matrix[int(i.Index),int(j.Index)] = 1.0
                    routeassignment_list[int(i.Index)].append(int(j.Index))
                    
            if self.config.OBJECTIVE == "time":
                cost_matrix[int(i.Index)] = float(i.total_delivery_time)
                for sj, j in enumerate(self.prob.shopper.itertuples()):
                    routeshopper_cost_matrix[int(i.Index), int(j.Index)] = driving_time[si, sj] + picking_time[si, sj]
            elif self.config.OBJECTIVE == "distance":
                cost_matrix[int(i.Index)] = float(i.total_distance)
                for sj, j in enumerate(self.prob.shopper.itertuples()):
                    routeshopper_cost_matrix[int(i.Index), int(j.Index)] = self._d[si, sj]
        
        m.RouteOrderMatrix = pe.Set(initialize=routeorder_matrix.keys())
        m.RouteShopperMap = pe.Set(initialize=routeshopper_cost_matrix.keys())
        m.RouteAssignmentMap = pe.Set(initialize=routeassignment_matrix.keys())
        
        # Non-generic pyomo mapping
        m._RouteOrderMap = routeorder_mapping
        m._RouteAssignmentList = routeassignment_list
        
        m.a = pe.Param(m.RouteOrderMatrix, initialize=routeorder_matrix)
        m.c = pe.Param(m.ColumnRange, initialize=cost_matrix)
        m.d = pe.Param(m.RouteShopperMap, initialize=routeshopper_cost_matrix)
        m.e = pe.Param(m.RouteAssignmentMap, initialize=routeassignment_matrix)
        
        pass
    
    def _pyomo_post_model(self, m):
        
        m.x = pe.Var(m.ColumnRange, within=pe.Binary)
        m.y = pe.Var(m.RouteAssignmentMap, within=pe.Binary)
        
        def post_objective(m):
            return sum(m.c[i] * m.x[i] for i in m.ColumnRange) + sum(m.d[i,j] * m.y[i,j] for (i,j) in m.RouteAssignmentMap)
        
        def constr_delivery_once(m, k):
            return sum(m.x[j] for j in m._RouteOrderMap[k]) == 1
    
        def constr_assign_to_shopper(m, i):
            return sum(m.y[i,j] for j in m._RouteAssignmentList[i]) >= m.x[i]
            
        def constr_assign_route_once(m, j):
            return sum(m.y[i,j] for i in m.ColumnRange if j in m._RouteAssignmentList[i]) <= 1
        
        m.objective = pe.Objective(rule=post_objective, sense=pe.minimize)
        m.constr_delivery_once = pe.Constraint(m.OrderRange, rule=constr_delivery_once)
        m.constr_assign_to_shopper = pe.Constraint(m.ColumnRange, rule=constr_assign_to_shopper)
        m.constr_assign_route_once = pe.Constraint(m.ShopperRange, rule=constr_assign_route_once)
        
        pass
    
    def _filter_route_time_window(self):
        """ 
            filter time infeasible routes based on each whether shopper is able to arrive on time 
            early is allowed
        """

        df = self.columns.copy()  # fetch the entire column sheet
        keep_cols = []
        total_drop_cnt = 0

        for cnt, i in enumerate(self.prob.shopper.itertuples()):
            s = i.Index
            df["infea_%s" % s] = False  # assume all shoppers can perform the delivery
            keep_cols.append("infea_%s" % s)
            if self.config.RELAX_TIME_WINDOW:
                continue
            
            df["t_forward"] = self.prob.plan.snapshot_at            
            df["t_forward"] += pd.to_timedelta(self.DDT["driving_time_%s" % s], unit='m')
            df["t_forward"] += pd.to_timedelta(self.DPT["picking_time_%s" % s], unit='m')
            
            # Sweeping forward for infeasible time windows
            for j in range(1, self.config.MAX_BATCH_SIZE+1):
                df["t_forward"] += pd.to_timedelta(df["delivery_time_%s" % j], unit='m')
                df["infea_%s" % s] = (df["t_forward"] > df["due_%s" % j]) | df["infea_%s" % s] # earliest arrive check
                
            # Sweeping backward for infeasible time windows
            for j in range(self.config.MAX_BATCH_SIZE, 1, -1):
                df["t_backward"] = df["due_%s" % j] - pd.to_timedelta(df["delivery_time_%s" % j], unit='m')
                df["infea_%s" % s] = (df["t_backward"] < df["open_%s" % (j-1)]) | df["infea_%s" % s] # latest arrival check
            
            to_drop_cnt = df["infea_%s" % s].sum()
            to_drop_perc = round(float(to_drop_cnt)/len(df)*100.0, 3)
            total_drop_cnt += to_drop_cnt
            
            # print "%s /%s (%s %%) dropped by %s-th shopper (%s) [time window infeasible]" % (to_drop_cnt, len(df), to_drop_perc, cnt, s)
            df = df.drop(columns=["t_forward", "t_backward"])
        
        print "Eliminated %s y-variables" % total_drop_cnt
        df = df.drop(columns=[i for i in df.columns if i not in keep_cols])
                                
        return df
    
    def _filter_inference_cleaning(self):        
        """ Use simple inference to eliminate more columns """
        check_vals = self.AssignableShopperMatrix.values.sum(axis=1)
        routes_to_drop = pd.Series(check_vals == len(self.prob.shopper), index=self.columns.index)
        self.columns = self.columns[~routes_to_drop]
        self.DDT = self.DDT[~routes_to_drop]
        self.DPT = self.DPT[~routes_to_drop]
        self.AssignableShopperMatrix = self.AssignableShopperMatrix[~routes_to_drop]
        
        pass
    
    def _other_ugly_misc_steps(self):
        self.AssignableShopperMatrix = np.invert(self.AssignableShopperMatrix).values  # True means keeping it
        pass

# Usage and Options
* use `problem()` object to construct a optimization problem based on a `plan`
* solve the problem using `LocalAlgorithm`
* applicable OPTIONS are controlled through `user_config`:
    * `MAX_BATCH_SIZE`: integer indicating largest size of a batch
    * `PACKAGE`: choice of modeling packages, support `"pyomo"` and `"pulp"`
    * `MIP_SOLVER`: choice of solver for MIP, support `"cbc"` and `"cplex"` (installation required)
    * `OBJECTIVE`: choice of objective function, support `"time"` and `"distance"`
    * `RELAX_TIME_WINDOW`: as the algorithm to ignore time window assumption or not
    * `ENUMERATOR`: choice of which enumeration function to call when generating batch combinations, support function `combinations` and `permutations`
* Call `cell.solve()` to solve a function after algorithm cell is loaded

In [12]:
%%time
prob = problem(plan)
cell = LocalAlgorithm(prob, user_config={"MAX_BATCH_SIZE": 2,
                                         "PACKAGE":"pyomo",
                                         "MIP_SOLVER":"cplex",
                                         "OBJECTIVE":"time",
                                         "RELAX_TIME_WINDOW":False,
                                         "ENUMERATOR":permutations})
cell.solve()

66 orders | 66 shoppers | 17 warehouse
generation before processing 1194
[0.01] initialize routes
[0.02] routes distances
[0.03] routes misc 
[0.14] route delivery time
[0.01] route presumed driving time
247 (21.0 %) routes dropped [obvious time window infeasible]
[0.0] routes filter R1
[0.14] d matrix df
[7.08] DPT matrix
[3.69] DDT matrix
Eliminated 12631 y-variables
[1.34] time window filter
[0.0] inference filter
[0.0] ugly steps...
sending 763 routes to solver
took 3.75995492935 seconds to construct mip model
-------------
mip status ok
objective = 3259.79
took 5.58372616768 seconds to solve mip model
CPU times: user 17.3 s, sys: 384 ms, total: 17.6 s
Wall time: 22 s


# Visualization and Results
* triangles are shoppers
* rectangles are warehouses
* dots are deliveries

In [13]:
### Visualization part
m = folium.Map(location=[37.9741, -122.541], zoom_start = 11, 
               tiles = 'Mapbox', API_key = "instacart.n1nj85bg", 
               width = 1000, height = 1000, detect_retina=True)

for index, row in cell.prob.order.iterrows():
    w_id = int(row.warehouse_id)
    m.add_child(folium.CircleMarker([row.end_lat, row.end_lon], color=Paired12[w_id % 12], radius=3))

for index, row in cell._output.iterrows():
    w_id = int(row.warehouse_id)
    m.add_child(folium.RegularPolygonMarker([row.lat_0, row.lon_0], 
                                            number_of_sides=4, 
                                            color=Paired12[w_id % 12], 
                                            fill_color=Paired12[w_id % 12], 
                                            radius=8))
    m.add_child(folium.RegularPolygonMarker([row.available_latitude, row.available_longitude],
                                            radius=5,
                                            rotation=np.arctan((row.lon_0-row.available_longitude)/(row.lat_0-row.available_latitude)),
                                            number_of_sides=3,
                                            color=Paired12[w_id % 12],
                                            fill_color=Paired12[w_id % 12]))
    
for i in cell._output.itertuples():
    route_points = [(i.available_latitude, i.available_longitude)]
    for j in range(cell.config.MAX_BATCH_SIZE+1):
        if not np.isnan(getattr(i, "lat_%s" % j)) and not np.isnan(getattr(i, "lon_%s" % j)):
            route_points.append((getattr(i, "lat_%s" % j), getattr(i, "lon_%s" % j)))
    w_id = i.warehouse_id
    folium.PolyLine(route_points, color=Paired12[w_id % 12], weight=1.5, opacity=1).add_to(m)
m